In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
df=pd.read_csv('imdb_movies.csv')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10178 entries, 0 to 10177
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   names       10178 non-null  object 
 1   date_x      10178 non-null  object 
 2   score       10178 non-null  float64
 3   genre       10093 non-null  object 
 4   overview    10178 non-null  object 
 5   crew        10122 non-null  object 
 6   orig_title  10178 non-null  object 
 7   status      10178 non-null  object 
 8   orig_lang   10178 non-null  object 
 9   budget_x    10178 non-null  float64
 10  revenue     10178 non-null  float64
 11  country     10178 non-null  object 
dtypes: float64(3), object(9)
memory usage: 954.3+ KB


In [25]:
df.shape

(10178, 12)

In [26]:
df.head()

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
0,Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Creed III,Released,English,75000000.0,2.716167e+08,AU
1,Avatar: The Way of Water,12/15/2022,78.0,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU
2,The Super Mario Bros. Movie,04/05/2023,76.0,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...","Chris Pratt, Mario (voice), Anya Taylor-Joy, P...",The Super Mario Bros. Movie,Released,English,100000000.0,7.244590e+08,AU
3,Mummies,01/05/2023,70.0,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...","Óscar Barberán, Thut (voice), Ana Esther Albor...",Momias,Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU
4,Supercell,03/17/2023,61.0,Action,Good-hearted teenager William always lived in ...,"Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quin...",Supercell,Released,English,77000000.0,3.409420e+08,US


CLEANING THE DATA

In [32]:
df.isnull().sum()

names          0
date_x         0
score          0
genre         85
overview       0
crew          56
orig_title     0
status         0
orig_lang      0
budget_x       0
revenue        0
country        0
dtype: int64

In [28]:
df['genre'].value_counts()

genre
Drama                                                   556
Comedy                                                  373
Drama, Romance                                          268
Horror                                                  260
Horror, Thriller                                        202
                                                       ... 
Action, Animation, Crime, Drama                           1
Adventure, Animation, Family, Action                      1
Drama, Animation, Family, Comedy, Fantasy                 1
Science Fiction, War                                      1
Action, Adventure, Science Fiction, Thriller, Horror      1
Name: count, Length: 2303, dtype: int64

In [29]:
df['genre'].unique()

array(['Drama,\xa0Action', 'Science Fiction,\xa0Adventure,\xa0Action',
       'Animation,\xa0Adventure,\xa0Family,\xa0Fantasy,\xa0Comedy', ...,
       'Adventure,\xa0Comedy,\xa0Family,\xa0Science Fiction,\xa0Action',
       'Mystery,\xa0Thriller,\xa0Comedy',
       'Action,\xa0Adventure,\xa0Science Fiction,\xa0Thriller,\xa0Horror'],
      dtype=object)

In [30]:
df['genre'] = df['genre'].str.replace('\xa0', ' ', regex=True)
df['genre'].unique()

array(['Drama, Action', 'Science Fiction, Adventure, Action',
       'Animation, Adventure, Family, Fantasy, Comedy', ...,
       'Adventure, Comedy, Family, Science Fiction, Action',
       'Mystery, Thriller, Comedy',
       'Action, Adventure, Science Fiction, Thriller, Horror'],
      dtype=object)

In [36]:
df = df.dropna(subset=['genre'])
df = df.dropna(subset=['crew'])

In [68]:
df.isnull().sum()
from scipy.stats import zscore
df['z_score'] = zscore(df['budget_x'])
outliers = df[df['z_score'].abs() > 3]
from scipy.stats import zscore
df['z_score'] = zscore(df['revenue'])
revenue = df[df['z_score'].abs() > 3]

In [54]:
df.dtypes

names          object
date_x         object
score         float64
genre          object
overview       object
crew           object
orig_title     object
status         object
orig_lang      object
budget_x      float64
revenue       float64
country        object
z_score       float64
dtype: object

In [57]:
df.head(1)

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country,z_score
0,Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Creed III,Released,English,75000000.0,271616668.0,AU,0.191944


In [62]:
df['date_x'] = pd.to_datetime(df['date_x'], errors='coerce')
df['genre'] = df['genre'].astype('category')

In [65]:
has_nat = df['date_x'].isna().any()
has_nat

np.False_

In [67]:
df.dtypes

names                 object
date_x        datetime64[ns]
score                float64
genre               category
overview              object
crew                  object
orig_title            object
status                object
orig_lang             object
budget_x             float64
revenue              float64
country               object
z_score              float64
dtype: object

In [69]:
revenue

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country,z_score
1,Avatar: The Way of Water,2022-12-15,78.0,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU,7.469523
62,Top Gun: Maverick,2022-05-26,83.0,"Action, Drama",After more than thirty years of service as one...,"Tom Cruise, Capt. Pete 'Maverick' Mitchell, Mi...",Top Gun: Maverick,Released,English,170000000.0,1.481369e+09,AU,4.448483
68,Avatar,2009-12-17,76.0,"Action, Adventure, Fantasy, Science Fiction","In the 22nd century, a paraplegic Marine is di...","Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar,Released,English,237000000.0,2.923706e+09,AU,9.664216
76,Spider-Man: No Way Home,2021-12-16,80.0,"Action, Adventure, Science Fiction",Peter Parker is unmasked and no longer able to...,"Tom Holland, Peter Parker / Spider-Man, Zenday...",Spider-Man: No Way Home,Released,English,200000000.0,1.910048e+09,AU,5.998658
104,Avengers: Infinity War,2018-04-25,83.0,"Adventure, Action, Science Fiction",As the Avengers and their allies have continue...,"Robert Downey Jr., Tony Stark / Iron Man, Chri...",Avengers: Infinity War,Released,English,300000000.0,2.048360e+09,AU,6.498816
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9457,Out of the Clear Blue Sky,2019-12-24,82.0,"Comedy, Drama, Music","Returning to Earth as an imitator, the legenda...","Omar Chaparro, , Ana Claudia Talancón, , Steph...",Como caído del cielo,Released,"Spanish, Castilian",144000000.0,1.135762e+09,MX,3.198710
9582,Charlton Heston: Radical to Right Wing,2023-02-24,90.0,"Documentary, TV Movie",A look at the life and work of the iconic US a...,"Maud Guillaumin, Self - Narrator (voice), Jona...",Charlton Heston : la démesure d'un géant,Released,French,264000000.0,1.263824e+09,FR,3.661802
9897,Female Market: Imprisonment,1986-01-18,49.0,Horror,Several women are kidnapped and treated to deg...,"Kaori Asô, Miki Uchiyama, Minako Ogawa, Akiko ...",凌辱めす市場 監禁,Released,Japanese,93000000.0,1.256345e+09,JP,3.634758
9969,Folklore: The Long Pond Studio Sessions,2020-11-25,86.0,"Music, Documentary","An intimate concert film, in which Taylor Swif...","Taylor Swift, Self, Jack Antonoff, Self, Aaron...",Folklore: The Long Pond Studio Sessions,Released,English,221000000.0,1.104642e+09,US,3.086173
